In [7]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve

# part a
St = 2775
delta_t = 1
sigma = 0.15
r = 0.0278
y = 0.0189
seq = [10**3, 10**4, 10**5, 10**6]
rand_dict = {}
for n in seq:
    np.random.seed(123)
    rand_dict[n] = np.random.normal(size=n)
# pseudo generate ST using the random series
def simulate_price(x, St=2775, delta_t=1, sigma=0.15, r=0.0278, y=0.0189):
    return St * np.exp((r - y - sigma**2 / 2) * delta_t + sigma * delta_t * x)
ST = {}
for key in rand_dict:
    ST[key] = simulate_price(rand_dict[key])
ST

{1000: array([2352.44722043, 3215.24179798, 2888.52942145, 2208.59494116,
        2538.3409781 , 3546.69515544, 1923.79319736, 2595.97937234,
        3347.42430525, 2430.9683766 , 2500.44277066, 2729.43438257,
        3462.56343221, 2515.48454669, 2590.11804852, 2593.86244373,
        3854.30328171, 3843.25114427, 3218.47882031, 2933.59521618,
        3092.27180758, 3462.22190285, 2405.90388074, 3302.48463186,
        2293.82022243, 2515.91869172, 3172.01339474, 2234.45788627,
        2710.92644792, 2432.78699779, 2664.34391514, 1819.41019772,
        2122.44936268, 2492.5821022 , 3181.71421211, 2697.31111857,
        2769.66849068, 3069.55976982, 2426.30687256, 2888.81055797,
        2453.45140386, 2136.46014819, 2610.82373321, 3017.32786429,
        2912.72510405, 2763.57787856, 3963.61123241, 2945.37920536,
        3206.27923521, 3872.97229824, 2280.02851331, 2369.03456513,
        3596.12753209, 2456.14079823, 2780.84053228, 3250.1402284 ,
        3164.22039922, 3602.16003228, 3464

In [8]:
# compute sample statistics
def sample_stats(sample_dict, max_min=False, output=[]):
    stats_df = pd.DataFrame(columns=seq)
    for key in sample_dict:
        stats_df.loc['mean', key] = sample_dict[key].mean()
        stats_df.loc['std dev', key] = sample_dict[key].std()
        stats_df.loc['log mean', key] = np.log(sample_dict[key]).mean()
        stats_df.loc['log variance', key] = np.log(sample_dict[key]).var()
        if max_min:
            stats_df.loc['max', key] = max(sample_dict[key])
            stats_df.loc['min', key] = min(sample_dict[key])
    if not output:
        return stats_df
    else:
        return stats_df.loc[output, :]

x_stats = sample_stats(ST)
x_stats

,1000,10000,100000,1000000
mean,2783.24,2803.78,2800.34,2800.09
std dev,419.211,422.194,422.323,422.488
log mean,7.92012,7.92751,7.92625,7.92615
log variance,0.0225355,0.0224152,0.0224807,0.0225116


In [13]:
def transformation(params, *args):
    x, St, delta_t, sigma, r, y, mean, var = args
    xp = params[0] + params[1] * x
    ST_p = simulate_price(xp)
    return [ST_p.mean() - mean, np.log(ST_p).var() - var]

ST_p = {}
for key in ST:
    args = (rand_dict[key], St, delta_t, sigma, r, y, x_stats.loc['mean', key], x_stats.loc['log variance', key])
    rlt = fsolve(transformation, [0.1, 0.9], args=args)
    print("Sequence {%s} generated result of: %s" % (key, rlt))
    xp = rlt[0] + rlt[1] * rand_dict[key]
    ST_p[key] = simulate_price(xp)
ST_p

Sequence {1000} generated result of: [1.24435255e-15 1.00000000e+00]
Sequence {10000} generated result of: [7.52932498e-16 1.00000000e+00]
Sequence {100000} generated result of: [4.14000828e-18 1.00000000e+00]
Sequence {1000000} generated result of: [-3.05148054e-15  1.00000000e+00]


{1000: array([2352.44722043, 3215.24179798, 2888.52942145, 2208.59494116,
        2538.3409781 , 3546.69515544, 1923.79319736, 2595.97937234,
        3347.42430525, 2430.9683766 , 2500.44277066, 2729.43438257,
        3462.56343221, 2515.48454669, 2590.11804852, 2593.86244373,
        3854.30328171, 3843.25114427, 3218.47882031, 2933.59521618,
        3092.27180758, 3462.22190285, 2405.90388074, 3302.48463186,
        2293.82022243, 2515.91869172, 3172.01339474, 2234.45788627,
        2710.92644792, 2432.78699779, 2664.34391514, 1819.41019772,
        2122.44936268, 2492.5821022 , 3181.71421211, 2697.31111857,
        2769.66849068, 3069.55976982, 2426.30687256, 2888.81055797,
        2453.45140386, 2136.46014819, 2610.82373321, 3017.32786429,
        2912.72510405, 2763.57787856, 3963.61123241, 2945.37920536,
        3206.27923521, 3872.97229824, 2280.02851331, 2369.03456513,
        3596.12753209, 2456.14079823, 2780.84053228, 3250.1402284 ,
        3164.22039922, 3602.16003228, 3464

In [14]:
# part b
def risk_neutral_put(ST, K=2775, delta_t=1, r=0.0278):
    return np.multiply(np.exp(- r * delta_t), [max(s, 0) for s in K - ST])

Pt = {}
Pt_p = {}
for key in ST: 
    Pt[key] = risk_neutral_put(ST[key])
    Pt_p[key] = risk_neutral_put(ST_p[key])

Pt_stats = sample_stats(Pt, output=['mean', 'std dev'])
Ptp_stats = sample_stats(Pt_p, output=['mean', 'std dev'])
print(Pt_stats, '\n', Ptp_stats)

         1000     10000    100000   1000000
mean      157.02  148.647  150.176  150.262
std dev  215.718  208.622  209.699  210.072 
          1000     10000    100000   1000000
mean      157.02  148.647  150.176  150.262
std dev  215.718  208.622  209.699  210.072


/Users/niniliu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys
/Users/niniliu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  
/Users/niniliu/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:112: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


In [15]:
def antithetic(x):
    x_anti = np.multiply(x, -1)
    ST = simulate_price(x)
    ST_anti = simulate_price(x_anti)
    PT = 0.5 * np.add(risk_neutral_put(ST), risk_neutral_put(ST_anti))
    return PT

P_anti = {}
for key in ST:
    P_anti[key] = antithetic(rand_dict[key])

Panti_stats = sample_stats(P_anti, output=['mean', 'std dev'])
Panti_stats

,1000,10000,100000,1000000
mean,149.861,150.418,150.399,150.373
std dev,104.938,103.318,103.657,103.822


In [16]:
# std error ratio
std_ratio = Pt_stats.loc['std dev', :] / Panti_stats.loc['std dev', :]

def efficiency_ratio(var1, var2, t1, t2):
    return var1 * t1 / (var2 * t2)

# pseudo generate ratio
t1 = 2
t2 = 1
eff_ratio = {}
for key in ST:
    eff_ratio[key] = efficiency_ratio(Panti_stats.loc['std dev', key]**2, Pt_stats.loc['std dev', key]**2, t1, t2)
eff_ratio

{1000: 0.47328349252234553,
 10000: 0.49052933327337467,
 100000: 0.4886966502922895,
 1000000: 0.48851021223290314}